In [1]:
import nest_asyncio
nest_asyncio.apply()

from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY")
)

Data directory: /Users/sali/Library/Application Support/crewai_storage


/Users/sali/miniconda3/envs/ML/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Data directory: /Users/sali/Library/Application Support/crewai_storage


In [2]:
from echo.runner import make_call
import asyncio
import nest_asyncio
nest_asyncio.apply()


test_clients = [
	"Shell",
	"Schneider electric"
]


train_clients = [
	"ICICI bank",
	"Infosys",	
	"University of Illinois",
	"Marks and spencer",
	"Mercedes Benz",
]

clients = test_clients + train_clients


inputs = {
    'call_type': 'discovery', 
    'seller': 'Whatfix',
    'n_competitors': 3
}

In [ ]:
discovery_calls_data = asyncio.run(make_call('discovery', clients, inputs))

In [4]:
from echo.step_templates.discovery import aget_simulation_data_for_client

simulation_data = await aget_simulation_data_for_client(inputs, llm)

<coroutine object aget_simulation_data_for_client at 0x3125c27a0>

In [ ]:
demo_calls_data = asyncio.run(make_call('demo', clients, inputs))

In [ ]:
pricing_call_data = asyncio.run(make_call('pricing', clients, inputs))

In [ ]:
negotiation_call_data = asyncio.run(make_call('negotiation', clients, inputs))

## Memory Module

## Using Memory

#### Memory Types

- Deal Summary

In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)

from llama_index.core.schema import TextNode
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
            "theme": "Fiction",
            "year": 2010,
        },
    ),
    TextNode(
        text="To Kill a Mockingbird",
        metadata={
            "author": "Harper Lee",
            "theme": "Mafia",
            "year": 1960,
        },
    ),
    TextNode(
        text="1984",
        metadata={
            "author": "George Orwell",
            "theme": "Totalitarianism",
            "year": 1949,
        },
    ),
    TextNode(
        text="The Great Gatsby",
        metadata={
            "author": "F. Scott Fitzgerald",
            "theme": "The American Dream",
            "year": 1925,
        },
    ),
    TextNode(
        text="Harry Potter and the Sorcerer's Stone",
        metadata={
            "author": "J.K. Rowling",
            "theme": "Fiction",
            "year": 1997,
        },
    ),
]

In [5]:
import enum
from typing import Dict
from llama_index.core.node_parser import SentenceSplitter
from echo.utils import db_storage_path

CHUNK_SIZE=8192
CHUNK_OVERLAP=128
SIMILARITY_TOP_K=3


class IndexType(enum.Enum):
    HISTORICAL = "historical"
    CURRENT_DEAL = "current_deal"
    BUYER_RESEARCH = "buyer_research"
    SELLER_RESEARCH = "seller_research"
    SALES_PLAYBOOK = "sales_playbook"


class CallType(enum.Enum):
    DISCOVERY = "discovery"
    DEMO = "demo"
    PRICING = "pricing"
    PROCUREMENT = "procurement"


def get_vector_index(
    index_name: str, 
    index_type: IndexType
):
    chroma_db_path = db_storage_path(index_name)
    db = chromadb.PersistentClient(path=str(chroma_db_path))
    chroma_collection = db.get_or_create_collection(f"{index_type.value}")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex.from_vector_store(vector_store)

    return index


def get_response(
    query: str, 
    index: VectorStoreIndex, 
    filters: Dict[str, str] = None, 
    filter_operator: FilterOperator = FilterOperator.EQ,
    condition: FilterCondition = FilterCondition.AND,
    
):
    filters = filters or {}
    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key=k.lower(), 
                value=v.lower(),
                operator=filter_operator
            ) for k, v in filters.items()
        ],
        condition=condition,
    )

    return index.as_query_engine(filters=filters).query(query)


def get_nodes_from_documents(data: str, metadata: Dict[str, str]):
    splitter = SentenceSplitter(
        chunk_size=CHUNK_SIZE, 
        chunk_overlap=CHUNK_OVERLAP
    )
    docs = splitter.split_text(data)
    return [
        TextNode(
            text=doc,
            metadata=metadata
        )
        for doc in docs
    ]

In [6]:
metadata_keys = {
    IndexType.HISTORICAL: [
        "seller", 
        "buyer", 
        "call_type", 
        "company_size", 
        "industry", 
        "description"
    ],
    IndexType.BUYER_RESEARCH: ["buyer"],
    IndexType.SELLER_RESEARCH: ["seller"],
    IndexType.SALES_PLAYBOOK: [],
}

def add_data(
    data: str, 
    metadata: Dict[str, str], 
    index_name: str, 
    index_type: IndexType
):
    assert all([k in metadata for k in metadata_keys[index_type]]), f"Missing metadata keys for {index_type}. \nRequired keys: {metadata_keys[index_type]}. \nProvided keys: {metadata.keys()}"
    filtered_metadata = {
        k.lower(): v.lower() for k, v in metadata.items() 
        if k in metadata_keys[index_type]
    }
    index = get_vector_index(index_name, index_type)
    nodes = get_nodes_from_documents(data, metadata=filtered_metadata)
    index.insert_nodes(nodes)

In [1]:
from echo.indexing import IndexType, get_vector_index
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters
)

from echo.settings import (
    SIMILARITY_TOP_K
)


class HistoricalIndexArguments(BaseModel):
    """Input schema for MyCustomTool."""
    query: str = Field(..., description="Query to retrieve from the index.")
    call_type: str = Field(..., description="Type of call")
    buyer: str = Field(..., description="Name of the buyer.")
    seller: str = Field(..., description="Name of the seller.")


class HistoricalCallIndex(BaseTool):
    name: str = "Historical Calls Vector Index"
    description: str = (
        "This tool retrieves relevant the historical calls data from the vector index for a given deal."
        "This tool uses buyer, seller and type to retrieve the relevant calls."
        "And then uses the query to retrieve the most relevant calls."
        "This tool can be used to see what has worked in the past for similar deals."
    )
    args_schema: Type[BaseModel] = HistoricalIndexArguments

    def _run(self, query: str, call_type: str, buyer: str, seller: str) -> str:
        index = get_vector_index(seller, IndexType.HISTORICAL)
        
        filters_dict = {
            "seller": seller,
            "buyer": buyer,
            "call_type": call_type
        }

        filters = MetadataFilters(
            filters=[
                MetadataFilter(
                    key=k, 
                    value=v,
                ) for k, v in filters_dict.items()
            ]
        )

        retrieved_nodes = index.as_retriever(filters=filters, similarity_top_k=SIMILARITY_TOP_K).retrieve(query)
        return f"\n".join([f"Historical Call: {i}\n{n.get_content()}" for i, n in enumerate(retrieved_nodes)])


class CurrentCallIndexArguments(BaseModel):
    """Input schema for MyCustomTool."""
    call_type: str = Field(..., description="Type of call")
    buyer: str = Field(..., description="Name of the buyer.")
    seller: str = Field(..., description="Name of the seller.")


class CurrentCallIndex(BaseTool):
    name: str = "Historical Calls Vector Index"
    description: str = (
        "This tool retrieves relevant documents related to the current call."
        "This tool uses buyer, seller and type to retrieve the relevant calls to retrieve the relevant call records."
    )
    args_schema: Type[BaseModel] = CurrentCallIndexArguments

    def _run(self, call_type: str, buyer: str, seller: str) -> str:
        index = get_vector_index(seller, IndexType.HISTORICAL)
        
        filters_dict = {
            "seller": seller,
            "buyer": buyer,
            "call_type": call_type
        }

        filters = MetadataFilters(
            filters=[
                MetadataFilter(
                    key=k, 
                    value=v,
                ) for k, v in filters_dict.items()
            ]
        )

        retrieved_nodes = index.as_retriever(filters=filters, similarity_top_k=SIMILARITY_TOP_K).retrieve("")
        return f"\n".join([f"Current {call_type} Call Data: {i}\n{n.get_content()}" for i, n in enumerate(retrieved_nodes)])

Data directory: /Users/sali/Library/Application Support/crewai_storage


/Users/sali/miniconda3/envs/ML/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Data directory: /Users/sali/Library/Application Support/crewai_storage


## Queries